
# 1. Imports


# 2. Carregando sparkDFs


## 2.1. Order payments

In [0]:
# Local do arquivo Delta
path_to_delta_table = "dbfs:/FileStore/Datum/KaggleOlistData/silver/delta/order_payments"

# Ler o arquivo Delta como um DataFrame
df_order_payments = spark.read.format("delta").load(path_to_delta_table)

In [0]:
display(df_order_payments.take(10))

order_id,payment_sequential,payment_type,payment_installments,payment_value
b81ef226f3fe1789b1e8b2acac839d17,1,credit_card,8,99.330000000000000000
a9810da82917af2d9aefd1278f1dcfa0,1,credit_card,1,24.390000000000000000
25e8ea4e93396b6fa0d3dd708e76c1bd,1,credit_card,1,65.710000000000000000
ba78997921bbcdc1373bb41e913ab953,1,credit_card,8,107.780000000000000000
42fdf880ba16b47b59251dd489d4441a,1,credit_card,2,128.450000000000000000
298fcdf1f73eb413e4d26d01b25bc1cd,1,credit_card,2,96.120000000000000000
771ee386b001f06208a7419e4fc1bbd7,1,credit_card,1,81.160000000000000000
3d7239c394a212faae122962df514ac7,1,credit_card,3,51.840000000000000000
1f78449c87a54faf9e96e88ba1491fa9,1,credit_card,6,341.090000000000000000
0573b5e23cbd798006520e1d5b4c6714,1,boleto,1,51.950000000000000000


In [0]:
del path_to_delta_table


# 3. Transformação


## 3.1. Order payments

In [0]:
df_order_payments = df_order_payments.filter(df_order_payments['payment_type'] != 'not_defined')

In [0]:
display(df_order_payments.take(10))

order_id,payment_sequential,payment_type,payment_installments,payment_value
b81ef226f3fe1789b1e8b2acac839d17,1,credit_card,8,99.330000000000000000
a9810da82917af2d9aefd1278f1dcfa0,1,credit_card,1,24.390000000000000000
25e8ea4e93396b6fa0d3dd708e76c1bd,1,credit_card,1,65.710000000000000000
ba78997921bbcdc1373bb41e913ab953,1,credit_card,8,107.780000000000000000
42fdf880ba16b47b59251dd489d4441a,1,credit_card,2,128.450000000000000000
298fcdf1f73eb413e4d26d01b25bc1cd,1,credit_card,2,96.120000000000000000
771ee386b001f06208a7419e4fc1bbd7,1,credit_card,1,81.160000000000000000
3d7239c394a212faae122962df514ac7,1,credit_card,3,51.840000000000000000
1f78449c87a54faf9e96e88ba1491fa9,1,credit_card,6,341.090000000000000000
0573b5e23cbd798006520e1d5b4c6714,1,boleto,1,51.950000000000000000


In [0]:
df_total_sales_by_payment_type = df_order_payments.groupBy("payment_type").sum("payment_value")

In [0]:
del df_order_payments

In [0]:
df_total_sales_by_payment_type = df_total_sales_by_payment_type.withColumnRenamed("sum(payment_value)", "total_sales")

In [0]:
display(df_total_sales_by_payment_type.take(10))

payment_type,total_sales
boleto,2869361.270000000000000000
credit_card,12542084.190000000000000000
voucher,379436.870000000000000000
debit_card,217989.790000000000000000



# 4. Exportando dados para Delta Lake

> Teste Prático -> "Grave os mesmos dados em formato Delta Lake para aproveitar as funcionalidades de versionamento e transações ACID."

In [0]:
df_total_sales_by_payment_type.write.format("delta").mode("overwrite").option("mergeSchema", "true").save(
    "dbfs:/FileStore/Datum/KaggleOlistData/gold/delta/total_sales_by_payment_type"
)

In [0]:
del df_total_sales_by_payment_type


# 5. Criando Delta Table

In [0]:
%sql

create database if not exists olist;

In [0]:
%sql

use olist;

In [0]:
%sql

drop table if exists total_sales_by_payment_type;

In [0]:
%sql

CREATE TABLE total_sales_by_payment_type
USING DELTA
LOCATION "dbfs:/FileStore/Datum/KaggleOlistData/gold/delta/total_sales_by_payment_type"